In [1]:
!pip install boto3

In [3]:
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54.9M  100 54.9M    0     0   125M      0 --:--:-- --:--:-- --:--:--  125M


In [ ]:
!unzip awscliv2.zip
!sudo ./aws/install

In [2]:
# load image from aws s3 bucket
import boto3
import tensorflow as tf
from PIL import Image
import numpy as np
s3 = boto3.resource('s3')
dataset = s3.Bucket('dalle2images')
image_paths = []

bucket_name = 'dalle2images'
for obj in dataset.objects.all():
    if obj.key.endswith('.png') and obj.key.startswith('real'):
        image_paths.append(obj.key)

    elif obj.key.endswith('.png') and obj.key.startswith('fake'):
        image_paths.append(obj.key)


print("Number of images: %d" % len(image_paths))

Number of images: 42878


In [13]:
bucket_name = 'dalle2images'
image = []
labels = []
for i in range(0,len(image_paths), 2):
    # read image from s3 bucket
    img = s3.Object(bucket_name, image_paths[i])
    img = Image.open(img.get()['Body']).resize([128,128])
    # convert image to numpy array
    img = np.array(img).astype(np.float32)
    # img = tf.convert_to_tensor(img, dtype=tf.float32)
    # add image to list
    image.append(img/255)
    if i < 21439:
      labels.append(0)
    else:
      labels.append(1)

print("Number of images: %d" % len(image))

Number of images: 21439


In [14]:
len(image)

21439

In [15]:
len(labels)

21439

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [16]:
#make labels into numpy array
alllabels = np.array(labels)
allimage = np.array(image).reshape(-1, 128, 128, 3)

In [18]:
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(allimage, alllabels, test_size=0.2, random_state=42)

# MobileV2 model

In [20]:
# create model using gpu
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.regularizers import l2


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add the data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

# Define the regularization strength
reg_strength = 0.001

# Create a new model using the base model and adding custom top layers
mobilemodel = tf.keras.models.Sequential([
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
mobilemodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 1s 0us/step


In [21]:
# Train the model for 10 epochs using the train dataset and avoid crashing
epochs = 10

mobilemodel.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
536/536 [==============================] - 23s 16ms/step - loss: 1.0599 - accuracy: 0.6394 - val_loss: 0.7807 - val_accuracy: 0.7575
Epoch 2/10
536/536 [==============================] - 6s 12ms/step - loss: 0.7656 - accuracy: 0.7165 - val_loss: 0.6512 - val_accuracy: 0.7516
Epoch 3/10
536/536 [==============================] - 6s 12ms/step - loss: 0.6624 - accuracy: 0.7337 - val_loss: 0.5817 - val_accuracy: 0.7868
Epoch 4/10
536/536 [==============================] - 6s 12ms/step - loss: 0.6199 - accuracy: 0.7366 - val_loss: 0.5286 - val_accuracy: 0.7852
Epoch 5/10
536/536 [==============================] - 6s 12ms/step - loss: 0.5948 - accuracy: 0.7409 - val_loss: 0.5208 - val_accuracy: 0.7803
Epoch 6/10
536/536 [==============================] - 6s 12ms/step - loss: 0.5801 - accuracy: 0.7453 - val_loss: 0.5162 - val_accuracy: 0.7889
Epoch 7/10
536/536 [==============================] - 7s 12ms/step - loss: 0.5757 - accuracy: 0.7446 - val_loss: 0.5201 - val_accuracy: 0.788

In [ ]:
pickle.dump(mobilemodel, open('mobilemodel.pkl', 'wb'))

In [3]:
# Evaluate the model
loss, accuracy = mobilemodel.evaluate(X_test, y_test)
print("Test accuracy: {:.2f}%".format(accuracy * 100))

NameError: name 'mobilemodel' is not defined

In [30]:
mobilemodel.predict(X_test[[0]])

1/1 [==============================] - 1s 825ms/step


array([[0.42874882]], dtype=float32)

## Hugging Face

In [45]:
# SaveModel format for tensorflow serving
mobilemodel.save('mobilemodel')

In [73]:
mobilemodel.build

<bound method Sequential.build of <keras.engine.sequential.Sequential object at 0x7f6a9050c3a0>>

In [75]:
from huggingface_hub.keras_mixin import push_to_hub_keras
push_to_hub_keras(mobilemodel, "mobilenet_v2_fake_image_detection")

fingerprint.pb:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

keras_metadata.pb:   0%|          | 0.00/554k [00:00<?, ?B/s]

saved_model.pb:   0%|          | 0.00/3.81M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

variables.data-00000-of-00001:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

'https://huggingface.co/Emmawang/mobilenet_v2_fake_image_detection/tree/main/'

In [4]:
from huggingface_hub import from_pretrained_keras

reloaded_model = from_pretrained_keras('Emmawang/mobilenet_v2_fake_image_detection')

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [11]:
import numpy as np
from PIL import Image
img = Image.open('/content/glasses_real.png').resize([128,128])
img = np.array(img).astype(np.float32)
img = img/255
img = img.reshape(-1, 128, 128, 3)
img.shape

(1, 128, 128, 3)

In [14]:
reloaded_model.predict(img)

1/1 [==============================] - 8s 8s/step


array([[0.69715124]], dtype=float32)

## Mapping

Facial Expression

In [24]:
import pandas as pd
df_csv = pd.read_csv('df_final (1).csv')

# delete rows after 21439
df_csv = df_csv.drop(df_csv.index[21439:])

In [39]:
df_csv.FacialExp.unique()

array(['neg', 'neutral', 'pos'], dtype=object)

In [40]:
import random

neg = []
for i in range(len(df_csv)):
    if df_csv['FacialExp'][i] == "neg":
        neg.append(i)

#randomly select 1000 images from neg
neg = random.sample(neg, 100)

# save the list of Unnamed: 0 that are pos Facial Expression
pos = []
for i in range(len(df_csv)):
    if df_csv['FacialExp'][i] == "pos":
        pos.append(i)
pos = random.sample(pos, 100)

# save the list of Unnamed: 0 that are neu Facial Expression
neu = []
for i in range(len(df_csv)):
    if df_csv['FacialExp'][i] == "neutral":
        neu.append(i)
neu = random.sample(neu, 100)

In [32]:
correct = 0

for i in range(len(neg)):
    img = s3.Object(bucket_name, image_paths[neg[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of negative expression is {correct/ len(neg) * 100} %")

1/1 [==============================] - 0s 25ms/step
the accuracy of negative expression is 75.0 %


In [33]:
correct = 0
for i in range(len(pos)):
    img = s3.Object(bucket_name, image_paths[pos[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of positive expression is {correct/ len(pos) * 100} %")

1/1 [==============================] - 0s 27ms/step
the accuracy of positive expression is 1.0 %
1/1 [==============================] - 0s 24ms/step
the accuracy of positive expression is 2.0 %
1/1 [==============================] - 0s 24ms/step
the accuracy of positive expression is 3.0 %
1/1 [==============================] - 0s 23ms/step
the accuracy of positive expression is 4.0 %
1/1 [==============================] - 0s 24ms/step
the accuracy of positive expression is 5.0 %
1/1 [==============================] - 0s 24ms/step
the accuracy of positive expression is 5.0 %
1/1 [==============================] - 0s 26ms/step
the accuracy of positive expression is 6.0 %
1/1 [==============================] - 0s 24ms/step
the accuracy of positive expression is 7.000000000000001 %
1/1 [==============================] - 0s 24ms/step
the accuracy of positive expression is 8.0 %
1/1 [==============================] - 0s 25ms/step
the accuracy of positive expression is 8.0 %
1/1 [===========

In [41]:
correct = 0
for i in range(len(neu)):
    img = s3.Object(bucket_name, image_paths[neu[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of neutral expression is {correct/ len(neu) * 100} %")

1/1 [==============================] - 0s 24ms/step
the accuracy of neutral expression is 70.0 %


In [46]:
man = []
for i in range(len(df_csv)):
    if df_csv['Gender'][i] == "Man":
        man.append(i)

#randomly select 1000 images from neg
man = random.sample(man, 100)

# save the list of Unnamed: 0 that are pos Facial Expression
woman = []
for i in range(len(df_csv)):
    if df_csv['Gender'][i] == "Woman":
        woman.append(i)
woman = random.sample(woman, 100)

In [50]:
correct = 0
for i in range(len(man)):
    img = s3.Object(bucket_name, image_paths[man[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of male is {correct/ len(man) * 100} %")

1/1 [==============================] - 0s 25ms/step
the accuracy of male is 83.0 %


In [49]:
correct = 0
for i in range(len(woman)):
    img = s3.Object(bucket_name, image_paths[woman[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of woman category is {correct/ len(woman) * 100} %")

1/1 [==============================] - 0s 24ms/step
the accuracy of woman category is 87.0 %


In [61]:
glasses = []
for i in range(len(df_csv)):
    if df_csv['Glasses'][i] == "1":
        glasses.append(i)
glasses = random.sample(glasses, 100)

In [60]:
noglasses = []
for i in range(len(df_csv)):
    if df_csv['Glasses'][i] == "0":
        noglasses.append(i)

noglasses = random.sample(noglasses, 100)


In [63]:
correct = 0
for i in range(len(glasses)):
    img = s3.Object(bucket_name, image_paths[glasses[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of human faces with glasses is {correct/ len(glasses) * 100} %")

1/1 [==============================] - 0s 25ms/step
the accuracy of human faces with glasses is 74.0 %


In [64]:
correct = 0
for i in range(len(noglasses)):
    img = s3.Object(bucket_name, image_paths[noglasses[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of human faces without glasses is {correct/ len(noglasses) * 100} %")

1/1 [==============================] - 0s 23ms/step
the accuracy of human faces without glasses is 78.0 %


In [66]:
white = []
for i in range(len(df_csv)):
    if df_csv['Race'][i] == "white":
        white.append(i)

white = random.sample(white, 100)


In [67]:
asian= []
for i in range(len(df_csv)):
    if df_csv['Race'][i] == "asian":
        asian.append(i)

asian = random.sample(asian, 100)


In [69]:
black= []
for i in range(len(df_csv)):
    if df_csv['Race'][i] == "black":
        black.append(i)

black = random.sample(black, 100)


In [70]:
correct = 0
for i in range(len(white)):
    img = s3.Object(bucket_name, image_paths[white[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of white category is {correct/ len(white) * 100} %")

1/1 [==============================] - 0s 24ms/step
the accuracy of white category is 80.0 %


In [71]:
correct = 0
for i in range(len(asian)):
    img = s3.Object(bucket_name, image_paths[asian[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of asian category is {correct/ len(asian) * 100} %")

1/1 [==============================] - 0s 24ms/step
the accuracy of asian category is 76.0 %


In [72]:
correct = 0
for i in range(len(black)):
    img = s3.Object(bucket_name, image_paths[black[i]])
    img = Image.open(img.get()['Body']).resize([128,128])
    img = np.array(img).astype(np.float32)
    img = img/255
    img = img.reshape(-1, 128, 128, 3)
    if mobilemodel.predict(img) < 0.5:
        correct += 1
print(f"the accuracy of black category is {correct/ len(black) * 100} %")

1/1 [==============================] - 0s 24ms/step
the accuracy of black category is 82.0 %
